In [103]:
import pickle

import numpy as np 
import pandas as pd

import re 
import string 

import nltk
#from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer 

from langdetect import detect

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
# from sklearn.metrics.pairwise import cosine_similarity

import spacy
from gensim import corpora, models, similarities, matutils

In [2]:
with open('pokemon.pickle','rb') as read_file:
    df = pickle.load(read_file)

### Data Cleaning


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2903 entries, 0 to 887
Data columns (total 5 columns):
name      2903 non-null object
date      2903 non-null object
rating    2903 non-null object
review    2903 non-null object
game      2903 non-null object
dtypes: object(5)
memory usage: 136.1+ KB


In [4]:
df['rating'] = df['rating'].astype(int)

In [5]:
### let's group the reviews by their ratings 
### following metacritics, 0 to 4 = negative, 5 to 7 mixed, 8 and above = positive 

def sentiment(x):
    if x > 7:
        return 'positive'
    if x < 5:
        return 'negative'
    else: return 'mixed'

df['sentiment'] = df['rating'].apply(lambda x:sentiment(x))


In [6]:
df['sentiment'].value_counts(normalize = True)

negative    0.610059
positive    0.306235
mixed       0.083707
Name: sentiment, dtype: float64

In [7]:
### Some users posted on both pokemon sword and shield. 
### Did some checks, these reviews were the same  

df[df.duplicated('name')]

,name,date,rating,review,game,sentiment
0,Metagrass,"Nov 15, 2019",2,"I have also done a review for Pokemon Sword, b...",shield,negative
1,NintendoGuy64,"Nov 15, 2019",0,"As a lifelong fan of Pokemon games, I was ecst...",shield,negative
4,Otonaburu,"Nov 15, 2019",4,What should have been a giant leap to signific...,shield,negative
7,Gamermangamer,"Nov 15, 2019",1,"Was promised a game for ""long time fans of the...",shield,negative
8,Fumetic,"Nov 15, 2019",3,"As these are largely the same games, I have pa...",shield,negative
...,...,...,...,...,...,...
871,HollyS,"Nov 20, 2019",2,"Very short, bland and low quality Pokemon game...",shield,negative
875,Lawrence7,"Nov 20, 2019",10,For anyone debating whether they will like the...,shield,positive
877,sojasonk,"Nov 20, 2019",1,"Lazy writing, bad graphics, an absolute medioc...",shield,negative
879,KrakenOfPepsi,"Nov 20, 2019",4,Metacritic has a pretty small character limit ...,shield,negative


In [8]:
df.review[df['name'] == 'Mack_thge_Sack']

1313    I'm going to state my points and not my emotio...
887     I'm going to state my points and not my emotio...
Name: review, dtype: object

In [9]:
df.reset_index(inplace = True, drop = True)

In [10]:
#dropping duplicates 
df.drop_duplicates(subset='name', keep = 'first', inplace = True)

In [11]:
### cleaning the text 

def make_lower(text):
    return text.lower()

def remove_digits(text):
    return re.sub('\d', '', text)

def clean_text(text):
    text = make_lower(text)
    #text = remove_punctuation(text)
    text = remove_digits(text)
    return text

In [12]:
df['review'] = df['review'].apply(lambda x:clean_text(x))

In [13]:
#df['review'] = df['review'].apply(lambda x:remove_stopwords(str.split(x)))

In [14]:
### check for english words 

In [15]:
def language_detection(x): 
    result = detect(x)
    if result == 'en':
        return x 
    else: return np.NaN 
    
df['review'] = df['review'].apply(lambda x:language_detection(x))

In [16]:
df.dropna(inplace = True)

In [17]:
# lemmatizer=WordNetLemmatizer()

# def lemmatize_words(x):
#     lemmed_string = ''
#     for word in x.split():
#         lemmed_string = lemmed_string+' '+lemmatizer.lemmatize(word)  
#     return lemmed_string.lstrip()

sp = spacy.load('en_core_web_sm')

def lemmatize_words(x):
    text = sp(x)
    lemmed_string =''
    for word in text:
        if word.lemma_ == '-PRON-':
            word.lemma_ = word.orth_ # change the string representation
            word.lemma = word.orth #
        else: lemmed_string = lemmed_string+' '+word.lemma_
    return lemmed_string.lstrip()

In [18]:
df['review'] = df['review'].apply(lambda x:lemmatize_words(x))

In [19]:
def remove_punctuation(text):
    return re.sub('[%s]' % re.escape(string.punctuation), '', text) 

df['review'] = df['review'].apply(lambda x:remove_punctuation(x))

In [28]:
def remove_r(x):
    return x.replace('\r','')

df['review'] = df['review'].apply(lambda x:remove_r(x))  

In [30]:
def strip_whitespace(x):
    return x.strip()

df['review'] = df['review'].apply(lambda x:strip_whitespace(x))  

In [86]:
def remove_rare(x):
    for word in freq:
        if word in x:
            return x.replace(word,'')   

df['review'] = df['review'].apply(lambda x:remove_rare(x))  

In [82]:
### create list of rare words by filtering on word counts
freq = pd.DataFrame(df.review.str.split(expand=True).stack().value_counts())
freq = freq[freq<11]
freq.dropna(inplace = True)
freq.reset_index(inplace = True)
freq = freq['index'].to_list()

In [90]:
df.to_pickle('dfclean.pickle')

# with open('dfclean.pickle','rb') as read_file:
#     df = pickle.load(read_file)

In [109]:
#freq = pd.DataFrame(df.review.str.split(expand=True).stack().value_counts())

In [96]:
### remove stopwords text, using this method just incase I want to add more stopwords 
nltk_stop_words = nltk.corpus.stopwords.words('english')

stop_words =  list(spacy.lang.en.stop_words.STOP_WORDS)
stop_words.extend(['game','pokemon','pokémon'])

for word in stop_words:
    if word in stop_words: 
        continue
    else: stop_words.append(word)

for word in stop_words:
    no_punct = remove_punctuation(word)
    if no_punct not in stop_words: 
        stop_words.append(no_punct)

## Splitting df by sentiment 

In [97]:
negative = df[df['sentiment']=='negative']
mixed = df[df['sentiment']=='mixed']
positive = df[df['sentiment']=='postive']

In [98]:
negative

,name,date,rating,review,game,sentiment
0,ChocolateCrane,"Nov 18, 2019",4,let address the elephant in the room first th...,sword,negative
3,firstlovezombie,"Nov 15, 2019",2,i really love pokemon which be why be so crit...,sword,negative
4,FilthyActs88,"Nov 15, 2019",1,pokémon sword and shield be a game of pure wh...,sword,negative
5,Ninjasuite,"Nov 15, 2019",3,this be what happen if a company realize that ...,sword,negative
6,JustrzBustrz,"Nov 15, 2019",2,’ an amazing feat how soulless this game feel ...,sword,negative
...,...,...,...,...,...,...
2889,Gonz04,"Nov 20, 2019",0,this game be so disappointing i literally end...,shield,negative
2891,InTheMood,"Nov 20, 2019",1,this be by far the bad pokémon game i have eve...,shield,negative
2893,Dreadwolf85,"Nov 20, 2019",3,lazy game design pokemon still do not hit eac...,shield,negative
2897,Cotroneo,"Nov 20, 2019",2,if consider this game a successor to the spin ...,shield,negative


In [113]:
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,2),
                                   strip_accents = 'ascii', 
                                   token_pattern="\\b[a-z][a-z][a-z]+\\b")

In [115]:
doc_word = vectorizer.fit_transform(negative)
doc_word.shape

(6, 4)

In [101]:
pd.DataFrame(doc_word.toarray(), index=df.review, columns=vectorizer.get_feature_names()).head(10)

aa  aa good  aaa  \
review                                                                 
let address the elephant in the room first  the...   0        0    0   
copy of sword and shield come in the mail a bit...   0        0    0   
the game be meh not even count the pokemon what...   0        0    0   
i really love pokemon  which be why be so criti...   0        0    0   
pokémon sword and shield be a game of pure  whe...   0        0    0   
this be what happen if a company realize that c...   0        0    0   
’ an amazing feat how soulless this game feel c...   0        0    0   
i really wish the game be good  i do not leave ...   0        0    0   
the game be full of graphical glitch  be short ...   0        0    0   
pokemon sword and shield boast expansive landsc...   0        0    0   

                                                    aaa console  aaa crash  \
review                                                                       
let address the elephant in the room first  the...            0          0   
copy of sword and shield come in the mail a bit...            0          0   
the game be meh not even count the pokemon what...            0          0   
i really love pokemon  which be why be so criti...            0          0   
pokémon sword and shield be a game of pure  whe...            0          0   
this be what happen if a company realize that c...            0          0   
’ an amazing feat how soulless this game feel c...            0          0   
i really wish the game be good  i do not leave ...            0          0   
the game be full of graphical glitch  be short ...            0          0   
pokemon sword and shield boast expansive landsc...            0          0   

                                                    aaa deserve  \
review                                                            
let address the elephant in the room first  the...            0   
copy of sword and shield come in the mail a bit...            0   
the game be meh not even count the pokemon what...            0   
i really love pokemon  which be why be so criti...            0   
pokémon sword and shield be a game of pure  whe...            0   
this be what happen if a company realize that c...            0   
’ an amazing feat how soulless this game feel c...            0   
i really wish the game be good  i do not leave ...            0   
the game be full of graphical glitch  be short ...            0   
pokemon sword and shield boast expansive landsc...            0   

                                                    aaa developer  \
review                                                              
let address the elephant in the room first  the...              0   
copy of sword and shield come in the mail a bit...              0   
the game be meh not even count the pokemon what...              0   
i really love pokemon  which be why be so criti...              0   
pokémon sword and shield be a game of pure  whe...              0   
this be what happen if a company realize that c...              0   
’ an amazing feat how soulless this game feel c...              0   
i really wish the game be good  i do not leave ...              0   
the game be full of graphical glitch  be short ...              0   
pokemon sword and shield boast expansive landsc...              0   

                                                    aaa element  aaa fall  \
review                                                                      
let address the elephant in the room first  the...            0         0   
copy of sword and shield come in the mail a bit...            0         0   
the game be meh not even count the pokemon what...            0         0   
i really love pokemon  which be why be so criti...            0         0   
pokémon sword and shield be a game of pure  whe...            0         0   
this be what happen if a company realize that c...            0         0   


## LSA 

In [104]:
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.0785955 , 0.01366075])

In [105]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

,aa,aa good,aaa,aaa console,aaa crash,aaa deserve,aaa developer,aaa element,aaa fall,aaa ile,...,zoo,zoo money,zoom,zoom max,zooopia,zooopia characer,zubat,zubat battle,zubat play,zubat tentacool
component_1,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_2,-0.0,-0.0,-0.004,0.0,0.0,-0.0,-0.0,-0.001,0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0


In [106]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [107]:
display_topics(lsa, vectorizer.get_feature_names(), 15)


Topic  0
like, new, battle, area, feel, good, time, wild, look, animation, play, bad, story, wild area, jt

Topic  1
area, battle, feel, wild, like, wild area, gym, story, feel like, route, raid, character, fight, trainer, team


### NMF 

In [ ]:
nmf_model = NMF(2)
doc_topic = nmf_model.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, vectorizer.get_feature_names(), 15)

### LDA 

In [ ]:
corpus = matutils.Sparse2Corpus(doc_word)

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)

In [ ]:
lda.print_topics()